# Exploring Data Clustering

Approaches to explore:
- [Priciple Component Analysis](https://en.wikipedia.org/wiki/Principal_component_analysis): [scikit-learn](http://scikit-learn.org/stable/auto_examples/decomposition/plot_pca_3d.html)
- [Hierarchical Clustering](https://en.wikipedia.org/wiki/Hierarchical_clustering): [scikit-learn](http://scikit-learn.org/stable/modules/clustering.html) [DBSCAN](http://scikit-learn.org/stable/modules/clustering.html#dbscan)



### Notes on [DBSCAN](http://scikit-learn.org/stable/modules/clustering.html#dbscan):
There are two parameters to the algorithm, `min_samples` and `eps`, which define formally what we mean when we say dense. Higher `min_samples` or lower `eps` indicate higher density necessary to form a cluster.

More formally, we define a core sample as being a sample in the dataset such that there exist `min_samples` other samples within a distance of `eps`, which are defined as neighbors of the core sample. This tells us that the core sample is in a dense area of the vector space. A cluster is a set of core samples, that can be built by recursively by taking a core sample, finding all of its neighbors that are core samples, finding all of their neighbors that are core samples, and so on. A cluster also has a set of non-core samples, which are samples that are neighbors of a core sample in the cluster but are not themselves core samples. Intuitively, these samples are on the fringes of a cluster.

Any core sample is part of a cluster, by definition. Further, any cluster has at least min_samples points in it, following the definition of a core sample. For any sample that is not a core sample, and does have a distance higher than eps to any core sample, it is considered an outlier by the algorithm.

### Consider instead using [HDBSCAN](https://github.com/scikit-learn-contrib/hdbscan)

In [ ]:
import glob
import os
import numpy as np
import pandas as pd
np.set_printoptions(suppress=True)

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.resources import INLINE
output_notebook(resources=INLINE)

In [ ]:
import bokeh.plotting
import bokeh.models   # Legend, Range1d
import bokeh.layouts # gridplot
from bokeh.palettes import Colorblind7 as palette

In [ ]:
import datashader
import collections
import xarray
from datashader.bokeh_ext import InteractiveImage

## Get the data
### Load the SAS data

Begin by designating where the theoretical SAXS and SANS data files are located.

In [5]:
sas_dir = '/Users/schowell/scratch/mab_clustering/sascalc'
saxs_dir = 'xray'
sans_dir = 'neutron_D2Op_100'
sas_ext = '*.iq'
saxs_search = os.path.join(sas_dir, saxs_dir, sas_ext)
sans_search = os.path.join(sas_dir, sans_dir, sas_ext)
print(saxs_search)
print(sans_search)

/Users/schowell/scratch/mab_clustering/sascalc/xray/*.iq
/Users/schowell/scratch/mab_clustering/sascalc/neutron_D2Op_100/*.iq


In [6]:
pr_dir = '/Users/schowell/scratch/mab_clustering/pr'
pr_ext = '*.pr'
pr_search = os.path.join(pr_dir, pr_ext)
print(pr_search)

/Users/schowell/scratch/mab_clustering/pr/*.pr


In [7]:
saxs_files = glob.glob(saxs_search)
sans_files = glob.glob(sans_search)
pr_files = glob.glob(pr_search)
saxs_files.sort()
sans_files.sort()
pr_files.sort()
n_saxs = len(saxs_files)
n_sans = len(sans_files)
n_pr = len(pr_files)
print('Found {} SAXS data files'.format(n_saxs))
print('Found {} SANS data files'.format(n_sans))
print('Found {} P(r) data files'.format(n_pr))

Found 200 SAXS data files
Found 200 SANS data files
Found 200 P(r) data files


In [8]:
saxs_data = []

# load in the first data set to setup the q-mask
first_data = np.loadtxt(saxs_files[0])
q_mask = first_data[:, 0] <= 0.18  # only use data up to 0.18 1/A
first_data = first_data[q_mask]
saxs_data.append(first_data[1:, 1])  # do not use I(0), it is the same for every dataset

# load in the rest of the data
for saxs_file in saxs_files[1:]:
    x_data = np.loadtxt(saxs_file)
    x_data = x_data[q_mask]
    assert np.allclose(x_data[0, 1], first_data[0, 1]), 'ERROR: data not normalized to I(0)'
    assert np.allclose(x_data[:, 0], first_data[:, 0]), 'ERROR: data not on same Q-grid'
    saxs_data.append(x_data[1:, 1])

saxs_data = np.array(saxs_data)

In [9]:
sans_data = []

# load in the first data set to setup the q-mask
first_data = np.loadtxt(sans_files[0])
q_mask = first_data[:, 0] <= 0.18  # only use data up to 0.18 1/A
first_data = first_data[q_mask]
sans_data.append(first_data[1:, 1])  # do not use I(0), it is the same for every dataset

# load in the rest of the data
for sans_file in sans_files[1:]:
    n_data = np.loadtxt(sans_file)
    n_data = n_data[q_mask]
    assert np.allclose(n_data[0, 1], first_data[0, 1]), 'ERROR: data not normalized'
    assert np.allclose(n_data[:, 0], first_data[:, 0]), 'ERROR: data not on same Q-grid'
    sans_data.append(n_data[1:, 1])
    
sans_data = np.array(sans_data)

Store the Q-values

In [10]:
q_saxs = x_data[1:, 0]
q_sans = n_data[1:, 0]
print(q_saxs)    
print(q_sans)

[ 0.01  0.02  0.03  0.04  0.05  0.06  0.07  0.08  0.09  0.1   0.11  0.12
  0.13  0.14  0.15  0.16  0.17  0.18]
[ 0.01  0.02  0.03  0.04  0.05  0.06  0.07  0.08  0.09  0.1   0.11  0.12
  0.13  0.14  0.15  0.16  0.17  0.18]


In [11]:
np.allclose(q_saxs, q_sans)

True

### Load the P(r) data

In [12]:
pr_data_l = []
n_pr = len(pr_files)
n_r = np.empty(n_pr, dtype=int)

# load in all the data set
for i, pr_file in enumerate(pr_files):
    n_data = np.loadtxt(pr_file, delimiter=',', dtype=int)
    pr_data_l.append(n_data[:, 1])
    n_r[i] = len(n_data)
    
r_max = n_r.max()
pr_data = np.zeros([n_pr, r_max], dtype=int)
for i, n_data in enumerate(pr_data_l):
    pr_data[i, :len(n_data)] = n_data
    
#pr_data = np.array(sans_data)

In [13]:
pr_data[:3, :8]

array([[     0,  17067,  50223,  97762, 154904, 252311, 345445, 436965],
       [     0,  17068,  50248,  97863, 155068, 252631, 345934, 437560],
       [     0,  17068,  50252,  97875, 155072, 252639, 345932, 437629]])

## Visualize the data 

Create a dataframe for plotting purposes (each column will be a different I(Q) or P(r) curve)

In [14]:
saxs_df = pd.DataFrame(data=saxs_data.T)
saxs_cols = ['c{}'.format(c) for c in saxs_df.columns]
saxs_df.columns = saxs_cols
saxs_df['q'] = q_saxs
saxs_df.head()

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,c191,c192,c193,c194,c195,c196,c197,c198,c199,q
0,0.109218,0.109229,0.109209,0.109217,0.109084,0.109038,0.109170,0.109135,0.109232,0.109550,...,0.110665,0.110607,0.110457,0.110620,0.110608,0.109585,0.109249,0.108942,0.108934,0.01
1,0.084704,0.084742,0.084680,0.084712,0.084166,0.084051,0.084403,0.084331,0.084601,0.085502,...,0.089196,0.089016,0.088568,0.089056,0.089012,0.085599,0.084535,0.083617,0.083595,0.02
2,0.056446,0.056516,0.056426,0.056480,0.055315,0.055224,0.055581,0.055574,0.055870,0.056941,...,0.062915,0.062657,0.062060,0.062711,0.062631,0.057028,0.055449,0.054231,0.054202,0.03
3,0.034189,0.034278,0.034189,0.034219,0.032548,0.032572,0.032673,0.032809,0.032909,0.033509,...,0.039937,0.039704,0.039241,0.039748,0.039643,0.033441,0.031986,0.031104,0.031075,0.04
4,0.021314,0.021403,0.021332,0.021273,0.019586,0.019701,0.019539,0.019758,0.019584,0.019508,...,0.024270,0.024138,0.023966,0.024158,0.024050,0.019180,0.018416,0.018257,0.018223,0.05


In [15]:
sans_df = pd.DataFrame(data=sans_data.T)
sans_cols = ['c{}'.format(c) for c in sans_df.columns]
sans_df.columns = sans_cols
sans_df['q'] = q_sans
sans_df.head()

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,c191,c192,c193,c194,c195,c196,c197,c198,c199,q
0,0.109411,0.109422,0.109402,0.109405,0.109262,0.109220,0.109345,0.109317,0.109405,0.109710,...,0.110870,0.110813,0.110664,0.110825,0.110813,0.109758,0.109420,0.109115,0.109105,0.01
1,0.085308,0.085347,0.085285,0.085304,0.084723,0.084618,0.084953,0.084900,0.085144,0.086011,...,0.089863,0.089683,0.089237,0.089722,0.089678,0.086144,0.085071,0.084148,0.084120,0.02
2,0.057354,0.057425,0.057335,0.057374,0.056155,0.056070,0.056413,0.056424,0.056687,0.057725,...,0.063975,0.063715,0.063116,0.063767,0.063686,0.057848,0.056237,0.054996,0.054956,0.03
3,0.035127,0.035220,0.035129,0.035155,0.033417,0.033439,0.033539,0.033679,0.033753,0.034346,...,0.041099,0.040861,0.040393,0.040904,0.040796,0.034266,0.032754,0.031828,0.031786,0.04
4,0.022116,0.022212,0.022139,0.022091,0.020328,0.020436,0.020280,0.020489,0.020304,0.020241,...,0.025271,0.025132,0.024959,0.025151,0.025038,0.019826,0.018998,0.018798,0.018754,0.05


In [16]:
pr_df = pd.DataFrame(data=pr_data.T)
pr_cols = ['c{}'.format(c) for c in pr_df.columns]
pr_df.columns = pr_cols
r = np.arange(r_max)
pr_df['r'] = r
pr_df.head()

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,c191,c192,c193,c194,c195,c196,c197,c198,c199,r
0,0,0,0,0,0,0,1,0,0,0,...,2,1,1,1,2,0,0,0,0,0
1,17067,17068,17068,17067,17070,17066,17073,17069,17076,17079,...,17101,17100,17099,17098,17091,17066,17066,17066,17066,1
2,50223,50248,50252,50265,50216,50207,50226,50219,50231,50226,...,50324,50310,50304,50312,50303,50202,50198,50199,50200,2
3,97762,97863,97875,97883,97600,97613,97605,97608,97604,97633,...,97901,97878,97836,97866,97828,97547,97542,97555,97547,3
4,154904,155068,155072,155081,154428,154434,154439,154443,154385,154410,...,154899,154866,154798,154872,154806,154143,154154,154178,154185,4


In [17]:
q_range = q_saxs.min(), q_saxs.max()
pq_range = saxs_data.min(), saxs_data.max()
r_range = 0, r_max
pr_range = pr_data.min(), pr_data.max()

In [18]:
%%time
sans_canvas = datashader.Canvas(x_range=q_range, y_range=pq_range, plot_height=300, plot_width=300)
# create an ordered-dictionary of each line, aggregated
sans_aggs = collections.OrderedDict((c, sans_canvas.line(sans_df, 'q', c)) for c in sans_cols)
sans_merged = xarray.concat(sans_aggs.values(), dim=pd.Index(sans_cols, name='cols'))
sans_img = datashader.transfer_functions.shade(sans_merged.sum(dim='cols'), how='eq_hist')

KeyboardInterrupt: Failed at nopython (nopython frontend)
Failed at nopython (nopython mode backend)


In [ ]:
%%time
saxs_canvas = datashader.Canvas(x_range=q_range, y_range=pq_range, plot_height=300, plot_width=300)
# create an ordered-dictionary of each line, aggregated
saxs_aggs = collections.OrderedDict((c, saxs_canvas.line(saxs_df, 'q', c)) for c in saxs_cols)
saxs_merged = xarray.concat(saxs_aggs.values(), dim=pd.Index(saxs_cols, name='cols'))
saxs_img = datashader.transfer_functions.shade(saxs_merged.sum(dim='cols'), how='eq_hist')

In [ ]:
%%time
pr_canvas = datashader.Canvas(x_range=r_range, y_range=pr_range, plot_height=300, plot_width=300)
# create an ordered-dictionary of each line, aggregated
pr_aggs = collections.OrderedDict((c, pr_canvas.line(pr_df, 'r', c)) for c in pr_cols)
pr_merged = xarray.concat(pr_aggs.values(), dim=pd.Index(pr_cols, name='cols'))
pr_img = datashader.transfer_functions.shade(pr_merged.sum(dim='cols'), how='eq_hist')

In [19]:
def saxs_image(q_range, pq_range, w, h):
    saxs_canvas = datashader.Canvas(x_range=q_range, y_range=pq_range, 
                                    plot_height=h, plot_width=w)
    saxs_aggs = collections.OrderedDict((c, saxs_canvas.line(saxs_df, 'q', c)) for c in saxs_cols)
    saxs_merged = xarray.concat(saxs_aggs.values(), dim=pd.Index(saxs_cols, name='cols'))
    saxs_img = datashader.transfer_functions.shade(saxs_merged.sum(dim='cols'), how='eq_hist')
    return saxs_img

p = bokeh.plotting.figure(x_range=q_range, y_range=pq_range,
                          plot_width=400, plot_height=300, 
                          x_axis_label='Q (1/A)', y_axis_label='P(Q)')
InteractiveImage(p, saxs_image)

In [21]:
def sans_image(q_range, pq_range, w, h):
    sans_canvas = datashader.Canvas(x_range=q_range, y_range=pq_range, 
                                    plot_height=h, plot_width=w)
    sans_aggs = collections.OrderedDict((c, sans_canvas.line(saxs_df, 'q', c)) for c in sans_cols)
    sans_merged = xarray.concat(sans_aggs.values(), dim=pd.Index(sans_cols, name='cols'))
    sans_img = datashader.transfer_functions.shade(sans_merged.sum(dim='cols'), how='eq_hist')
    return sans_img

p = bokeh.plotting.figure(x_range=q_range, y_range=pq_range,
                          plot_width=400, plot_height=300, 
                          x_axis_label='Q (1/A)', y_axis_label='P(Q)')
InteractiveImage(p, sans_image)

In [ ]:
def pr_image(r_range, pr_range, w, h):
    pr_canvas = datashader.Canvas(x_range=r_range, y_range=pr_range, 
                                  plot_height=h, plot_width=w)
    pr_aggs = collections.OrderedDict((c, pr_canvas.line(pr_df, 'r', c)) for c in pr_cols)
    pr_merged = xarray.concat(pr_aggs.values(), dim=pd.Index(pr_cols, name='cols'))
    pr_img = datashader.transfer_functions.shade(pr_merged.sum(dim='cols'), how='eq_hist')
    return pr_img

p = bokeh.plotting.figure(x_range=r_range, y_range=pr_range,
                          plot_width=400, plot_height=300, 
                          x_axis_label='r (A)', y_axis_label='P(r)')
InteractiveImage(p, pr_image)

In [ ]:
sans_img

In [ ]:
saxs_img

In [ ]:
pr_img

### Manipulate the data

In [ ]:
n_samples, n_features = saxs_data.shape # for PCA, should be (n_samples, n_features)
print('# samples: {}\n# features: {}'.format(n_samples, n_features))

Each __row__ in the NumPy array is a separate scattering pattern or pair-distance distribution curve.  Each __column__ with be a feature we explore.

In [ ]:
print(saxs_data[:3, :5])
print(sans_data[:3, :5])
print(pr_data[:3, :5])

In [ ]:
min_vals = saxs_data.min(axis=0)
max_vals = saxs_data.max(axis=0)
saxs_range = max_vals - min_vals
print(saxs_range)

In [ ]:
min_vals = sans_data.min(axis=0)
max_vals = sans_data.max(axis=0)
sans_range = max_vals - min_vals
print(sans_range)

Notice how the variation depends significantly on Q.

In [ ]:
min_vals = pr_data.min(axis=0)
max_vals = pr_data.max(axis=0)
pr_range = max_vals - min_vals
print(pr_range)

In [ ]:
range_pq = bokeh.plotting.figure(x_axis_label='Q (1/A)', y_axis_label='P(Q) range',
                                 width=400, height=300, title='Data Range')
range_pr = bokeh.plotting.figure(x_axis_label='r (A)', y_axis_label='P(r) range',
                                 width=400, height=300, title='Data Range')

range_pq.line(q_saxs, saxs_range, legend='SAXS range', color=palette[0])
range_pq.line(q_sans, sans_range, legend='SANS range', color=palette[1])

range_pr.line(r, pr_range, legend='P(c) range', color=palette[3])
range_pr.legend.location = "bottom_center"

plots = bokeh.layouts.gridplot([[range_pq, range_pr]])
show(plots)

### Rescale the data
Originally used `StandardScaler` but changed to `RobustScaler` to avoid complications from outliers (which skew the mean)

In [ ]:
from sklearn.preprocessing import StandardScaler, RobustScaler
saxs_scaler = RobustScaler()
sans_scaler = RobustScaler()
pr_scaler = RobustScaler()

In [ ]:
saxs_scaler.fit(saxs_data)
sans_scaler.fit(sans_data)
pr_scaler.fit(pr_data)

In [ ]:
scaled_saxs = saxs_scaler.transform(saxs_data)
scaled_sans = sans_scaler.transform(sans_data)
scaled_pr = pr_scaler.transform(pr_data)
print(scaled_saxs[:3, :5])
print(scaled_sans[:3, :5])
print(scaled_pr[:3, :5])

In [ ]:
min_vals = scaled_saxs.min(axis=0)
max_vals = scaled_saxs.max(axis=0)
saxs_scaled_range = max_vals - min_vals

min_vals = scaled_sans.min(axis=0)
max_vals = scaled_sans.max(axis=0)
sans_scaled_range = max_vals - min_vals

min_vals = scaled_pr.min(axis=0)
max_vals = scaled_pr.max(axis=0)
pr_scaled_range = max_vals - min_vals

print(saxs_scaled_range)
print(sans_scaled_range)
print(pr_scaled_range)

Notice how now the range is essentially independent of Q or r.

In [ ]:
range_pq0 = bokeh.plotting.figure(x_axis_label='Q (1/A)', y_axis_label='P(Q) range',
                                 width=400, height=300, title='Range Before')
range_pr0 = bokeh.plotting.figure(x_axis_label='r (A)', y_axis_label='P(r) range',
                                 width=400, height=300, title='Comparison')
range_pq1 = bokeh.plotting.figure(x_axis_label='Q (1/A)', y_axis_label='P(Q) range',
                                 width=400, height=300, title='Comparison')
range_pr1 = bokeh.plotting.figure(x_axis_label='r (A)', y_axis_label='P(r) range',
                                 width=400, height=300, title='Range After')

range_pq0.line(q_saxs, saxs_range, legend='SAXS before', color=palette[0])
range_pq0.line(q_sans, sans_range, legend='SANS before', color=palette[1])
range_pq1.line(q_saxs, saxs_range, legend='SAXS before', color=palette[0])
range_pq1.line(q_sans, sans_range, legend='SANS before', color=palette[1])

range_pq1.line(q_saxs, saxs_scaled_range, legend='SAXS after', color=palette[0], line_dash= [4, 2])
range_pq1.line(q_sans, sans_scaled_range, legend='SANS after', color=palette[1], line_dash= [4, 2])


range_pr0.line(r, pr_range, legend='before', color=palette[3])
range_pr0.line(r, pr_scaled_range, legend='after', color=palette[3], line_dash= [4, 2])
range_pr1.line(r, pr_scaled_range, legend='after', color=palette[3], line_dash= [4, 2])

range_pr0.legend.location = "bottom_center"
range_pr1.legend.location = "top_left"

plots = bokeh.layouts.gridplot([[range_pq0, range_pr0],
                                [range_pq1, range_pr1]])
show(plots)

Note that for P(r), the last 5 points in almost every data set are zero, so the range did not change at all.  See this below

In [ ]:
pr_data[:, -5:]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def compare_q1_q2(q1, q2):
    plt.figure()
    plt.plot(saxs_data[:,q1], saxs_data[:,q2], 'bo', markersize=5)
    plt.title('original data')
    plt.xlabel(r'$Q_{}$'.format(q1))
    plt.ylabel(r'$Q_{}$'.format(q2))
    plt.figure()
    plt.plot(scaled_saxs[:,q1], scaled_saxs[:,q2], 'bo', markersize=5)
    plt.xlabel(r'$Q_{}$'.format(q1))
    plt.ylabel(r'$Q_{}$'.format(q2))
    plt.title('scaled data')

In [ ]:
compare_q1_q2(0, 8)

In [ ]:
plt.figure()
plt.plot(saxs_data[:,0], saxs_data[:,15], 'bo', markersize=5)
plt.title('original data')
plt.xlabel(r'$Q_{}$'.format(0))
plt.ylabel(r'$Q_{%d}$'%15)
plt.figure()
plt.plot(scaled_saxs[:,0], scaled_saxs[:,17], 'bo', markersize=5)
plt.xlabel(r'$Q_{}$'.format(0))
plt.ylabel(r'$Q_{}$'.format(17))
plt.title('scaled data')

In [ ]:
scaled_saxs.shape

In [ ]:
i0 = 2
i_compare = 0
for i0 in range(18):
    plt.figure()
    plt.plot(scaled_saxs[:,i0], scaled_saxs[:, i_compare], 'bo')
    plt.plot(scaled_saxs[112,i0], scaled_saxs[112, i_compare], 'rs')
    plt.plot(scaled_saxs[113,i0], scaled_saxs[113, i_compare], 'gs')
    plt.xlabel(r'$Q_{{}}$'.format(i0))
    plt.ylabel(r'$Q_{{}}$'.format(i_compare))

### DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn import metrics

In [ ]:
# Compute DBSCAN
## Tune these parameters to adjust cluster size ##
distance = 1  # <--need to explore tuning this parameter
min_samples = 2
##################################################
x_db = DBSCAN(eps=distance, min_samples=min_samples).fit(scaled_saxs)
x_core_samples_mask = np.zeros_like(x_db.labels_, dtype=bool)
x_core_samples_mask[x_db.core_sample_indices_] = True
saxs_labels = x_db.labels_ + 1 # 0's are independent groups
x_clusters_ = len(set(saxs_labels)) - (1 if -1 in saxs_labels else 0)

n_db = DBSCAN(eps=distance, min_samples=min_samples).fit(scaled_saxs)
n_core_samples_mask = np.zeros_like(n_db.labels_, dtype=bool)
n_core_samples_mask[n_db.core_sample_indices_] = True
sans_labels = n_db.labels_ + 1 # 0's are independent groups
n_clusters_ = len(set(sans_labels)) - (1 if -1 in sans_labels else 0)

p_db = DBSCAN(eps=distance, min_samples=min_samples).fit(scaled_pr)
p_core_samples_mask = np.zeros_like(p_db.labels_, dtype=bool)
p_core_samples_mask[p_db.core_sample_indices_] = True
pr_labels = p_db.labels_ + 1 # 0's are independent groups
p_clusters_ = len(set(pr_labels)) - (1 if -1 in pr_labels else 0)

In [ ]:
# x-ray clusters
x_unique = set(saxs_labels)
x_unique.remove(0)
print('cluster labels: {}'.format(x_unique))
print('unique clusters: {}'.format(len(x_unique) + list(saxs_labels).count(0)))
for c in set(saxs_labels):
    print('{}: {}'.format(c, list(saxs_labels).count(c)))

In [ ]:
# neutron clusters
unique = set(sans_labels)
unique.remove(0)
total_clusters = len(unique) + list(sans_labels).count(0)
print('cluster labels: {}'.format(unique))
print('unique clusters: {}'.format(total_clusters))
for c in set(sans_labels):
    print('{}: {}'.format(c, list(sans_labels).count(c)))

In [ ]:
# pr clusters
unique = set(pr_labels)
unique.remove(0)
total_clusters = len(unique) + list(pr_labels).count(0)
print('cluster labels: {}'.format(unique))
print('unique clusters: {}'.format(total_clusters))
for c in set(pr_labels):
    print('{}: {}'.format(c, list(pr_labels).count(c)))

In [ ]:
np.savetxt('saxs_clusters.txt', saxs_labels, fmt='%d')
np.savetxt('sans_clusters.txt', sans_labels, fmt='%d')
np.savetxt('pr_clusters.txt', pr_labels, fmt='%d')

In [ ]:
print(sans_labels)
print(sans_labels.shape)
slabels = np.array(sans_labels, dtype='str')
# print(slabels)
# print(slabels.shape)

In [ ]:
from matplotlib import offsetbox
i_compare = 0

mn = scaled_saxs.min(axis=0)
mx = scaled_saxs.max(axis=0)

# for i0 in range(1):
for i0 in range(18):
    plt.figure()
    
    # plot points to make the correct box size
    plt.plot(mn[i0], mn[i_compare], 'w.')
    plt.plot(mx[i0], mx[i_compare], 'w.')
    
    for j in range(len(scaled_saxs)):
        if slabels[j] != '0':
            plt.text(scaled_saxs[j, i0], scaled_saxs[j, i_compare], slabels[j],
                     fontdict={'weight': 'bold', 'size': 15}, 
                     color='r') # plt.cm.Set1(labels[i]/10.0))
        else:
            plt.plot(scaled_saxs[j, i0], scaled_saxs[j, i_compare], 'k.',
                    markersize=5)
                
    plt.xlabel(r'$Q_{}$'.format(i0))
    plt.ylabel(r'$Q_{}$'.format(i_compare))

### [HDBSCAN](http://hdbscan.readthedocs.io/en/latest/basic_hdbscan.html#the-simple-case)

In [ ]:
import hdbscan

In [ ]:
# Compute DBSCAN
## Tune these parameters to adjust cluster size ##
distance = 1  # <--need to explore tuning this parameter
min_samples = 2
##################################################
x_db = hdbscan.HDBSCAN(eps=distance, min_samples=min_samples).fit(scaled_saxs)
x_core_samples_mask = np.zeros_like(x_db.labels_, dtype=bool)
x_core_samples_mask[x_db.core_sample_indices_] = True
saxs_labels = x_db.labels_ + 1 # 0's are independent groups
x_clusters_ = len(set(saxs_labels)) - (1 if -1 in saxs_labels else 0)

n_db = DBSCAN(eps=distance, min_samples=min_samples).fit(scaled_saxs)
n_core_samples_mask = np.zeros_like(n_db.labels_, dtype=bool)
n_core_samples_mask[n_db.core_sample_indices_] = True
sans_labels = n_db.labels_ + 1 # 0's are independent groups
n_clusters_ = len(set(sans_labels)) - (1 if -1 in sans_labels else 0)

p_db = DBSCAN(eps=distance, min_samples=min_samples).fit(scaled_pr)
p_core_samples_mask = np.zeros_like(p_db.labels_, dtype=bool)
p_core_samples_mask[p_db.core_sample_indices_] = True
pr_labels = p_db.labels_ + 1 # 0's are independent groups
p_clusters_ = len(set(pr_labels)) - (1 if -1 in pr_labels else 0)

In [ ]:
# x-ray clusters
x_unique = set(saxs_labels)
x_unique.remove(0)
print('cluster labels: {}'.format(x_unique))
print('unique clusters: {}'.format(len(x_unique) + list(saxs_labels).count(0)))
for c in set(saxs_labels):
    print('{}: {}'.format(c, list(saxs_labels).count(c)))

In [ ]:
# neutron clusters
unique = set(sans_labels)
unique.remove(0)
total_clusters = len(unique) + list(sans_labels).count(0)
print('cluster labels: {}'.format(unique))
print('unique clusters: {}'.format(total_clusters))
for c in set(sans_labels):
    print('{}: {}'.format(c, list(sans_labels).count(c)))

In [ ]:
# pr clusters
unique = set(pr_labels)
unique.remove(0)
total_clusters = len(unique) + list(pr_labels).count(0)
print('cluster labels: {}'.format(unique))
print('unique clusters: {}'.format(total_clusters))
for c in set(pr_labels):
    print('{}: {}'.format(c, list(pr_labels).count(c)))

In [ ]:
np.savetxt('saxs_clusters.txt', saxs_labels, fmt='%d')
np.savetxt('sans_clusters.txt', sans_labels, fmt='%d')
np.savetxt('pr_clusters.txt', pr_labels, fmt='%d')

In [ ]:
print(sans_labels)
print(sans_labels.shape)
slabels = np.array(sans_labels, dtype='str')
# print(slabels)
# print(slabels.shape)

In [ ]:
from matplotlib import offsetbox
i_compare = 0

mn = scaled_saxs.min(axis=0)
mx = scaled_saxs.max(axis=0)

# for i0 in range(1):
for i0 in range(18):
    plt.figure()
    
    # plot points to make the correct box size
    plt.plot(mn[i0], mn[i_compare], 'w.')
    plt.plot(mx[i0], mx[i_compare], 'w.')
    
    for j in range(len(scaled_saxs)):
        if slabels[j] != '0':
            plt.text(scaled_saxs[j, i0], scaled_saxs[j, i_compare], slabels[j],
                     fontdict={'weight': 'bold', 'size': 15}, 
                     color='r') # plt.cm.Set1(labels[i]/10.0))
        else:
            plt.plot(scaled_saxs[j, i0], scaled_saxs[j, i_compare], 'k.',
                    markersize=5)
                
    plt.xlabel(r'$Q_{}$'.format(i0))
    plt.ylabel(r'$Q_{}$'.format(i_compare))

### Write DCD output

In [ ]:
import sasmol.sasmol as sasmol

dcd_fname = glob.glob('*.dcd')
assert len(dcd_fname) == 1, 'ERROR: unsure which dcd file to use: {}'.format(dcd_fname)
dcd_fname = dcd_fname[0]

pdb_fname = glob.glob('*.pdb')
assert len(pdb_fname) == 1, 'ERROR: unsure which dcd file to use: {}'.format(pdb_fname)
pdb_fname = pdb_fname[0]

In [ ]:
mol = sasmol.SasMol(0)
mol.read_pdb(pdb_fname)

In [ ]:
if not np.alltrue(sans_labels == saxs_labels):
    print('WARNING: labels do not match\nusing neutron labels')
labels = sans_labels

In [ ]:
dcd_fname

In [ ]:
# create a dcd for every cluster with >1 frame
dcd_fnames = []
cluster_out_files = [] # dcds for clusters
unique_out_fname = '{}_uniue.dcd'.format(dcd_fname[:-4]) 
dcd_out_file = mol.open_dcd_write(unique_out_fname) # dcd file for unique structures

dcd_in_file = mol.open_dcd_read(dcd_fname)

for i in xrange(len(unique)):
    dcd_fnames.append('{}_c{:02d}.dcd'.format(dcd_fname[:-4], i))
    cluster_out_files.append(mol.open_dcd_write(dcd_fnames[i]))

visited_cluster = set()
dcd_out_frame = 0
cluster_out_frame = np.zeros(len(unique), dtype=int)

for (i, label) in enumerate(labels):
    mol.read_dcd_step(dcd_in_file, i)
    if label == 0:
        dcd_out_frame += 1
        mol.write_dcd_step(dcd_out_file, 0, dcd_out_frame)
    else:
        cluster_out_frame[label-1] += 1
        # print('adding frame to cluster {}'.format(label-1))
        # print(cluster_out_frame)
        mol.write_dcd_step(cluster_out_files[label-1], 0, cluster_out_frame[label-1])
        if label not in visited_cluster:
            visited.add(label)
            dcd_out_frame += 1
            mol.write_dcd_step(dcd_out_file, 0, dcd_out_frame)
        
for cluster_out_file in cluster_out_files:
    mol.close_dcd_write(cluster_out_file)

mol.close_dcd_write(dcd_out_file)    
mol.close_dcd_read(dcd_in_file[0])


### PCA Analysis

In [ ]:
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
pca_orig = PCA()
pca_orig.fit(saxs_data)

In [ ]:
pca_scaled = PCA()
pca_scaled.fit(scaled_saxs)

In [ ]:
print(pca_orig.explained_variance_ratio_)
print(pca_scaled.explained_variance_ratio_)

In [ ]:
plt.figure()
plt.plot(q_values, pca_orig.explained_variance_ratio_, 'o', label='unscaled')
plt.plot(q_values, pca_scaled.explained_variance_ratio_, 's', label='scaled')
plt.legend()

In [ ]:
from sklearn.datasets.samples_generator import make_blobs

In [ ]:
centers = [[1, 1], [-1, -1], [1, -1]]
X, labels_true = make_blobs(n_samples=750, centers=centers, cluster_std=0.4,
                            random_state=0)

In [ ]:
X_scaled = StandardScaler().fit_transform(X)

In [ ]:
X_range = X.max(axis=0) - X.min(axis=0)
print(X_range)

In [ ]:
saxs_scaled_range = X_scaled.max(axis=0) - X_scaled.min(axis=0)
print(saxs_scaled_range)

In [ ]:
X_s2 = StandardScaler().fit_transform(X)

In [ ]:
X_s2_range = X_s2.max(axis=0) - X_s2.min(axis=0)
print(X_s2_range)